<a href="https://colab.research.google.com/github/UsernameLucky/Machine_Learning-projects/blob/main/NLP_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import random
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
#from torchtext.datasets import LanguageModelingDataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import os
import nltk
nltk.download('stopwords')
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
#from torchtext.data import Field, BucketIterator
import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#!pip install torch torchtext


# Load the files and concatenate them

We have 4 different txt files from 4 of Chinua Achebe's books; A Man of the People, Anthills of the Savannah, Arrow of God and Things Fall Apart that we need to concatenate to one dataset.



In [ ]:
data_path = '/content/drive/MyDrive/assignment_2_&_data'
files = [f for f in os.listdir(data_path) if f.endswith('.txt')]
texts = []
for file in files:
    with open(os.path.join(data_path, file), 'r', encoding='latin-1') as f:
       text = f.read()
    texts.append(text)
combined_text = ' '.join(texts)


We use the following code to check the first characters of the text.

In [ ]:
print(combined_text[:1000])  # Print the first 1000 characters

Anthills of the Savannah


Chinua Achebe


	Published 1988




ONE
_First Witness--Christopher Oriko__

'YOU'RE WASTING EVERYBODY'S TIME, Mr. Commissioner for Information. I will not go to Abazon. Finish! _Kabisa!__ Any other business?'
	'As Your Excellency wishes. But ...'
	'But me no buts, Mr. Oriko! The matter is closed, I said. How many times, for God's sake, am I expected to repeat it? Why do _you__ find it so difficult to swallow my ruling. On anything?'
	'I am sorry, Your Excellency. But I have no difficulty swallowing _and__ digesting your rulings.'
	For a full minute or so the fury of his eyes lay on me. Briefly our eyes had been locked in combat. Then I had lowered mine to the shiny table-top in ceremonial capitulation. Long silence. But he was not appeased. Rather he was making the silence itself grow rapidly into its own kind of contest, like the eyewink duel of children. I conceded victory there as well. Without raising my eyes I said again: 'I am very sorry, Your Excellen

The above code shows the first characters of our concatenated dataset. This happens to be Anthills of the Savannah.

In [ ]:
# Print the last 1000 characters
print(combined_text[-1000:])


e joint winner of the first Commonwealth Poetry Prize. Of his novels, Arrow of God is winner of the New Statesman-Jock Campbell Award, and Anthills of the Savannah was a finalist for the 1987 Booker Prize in England.
	Mr. Achebe has received numerous honours from around the world, including the Honorary Fellowship of the American Academy and Institute of Arts and Letters, as well as more than twenty honorary doctorates from universities in England, Scotland, the United States, Canada, and Nigeria. He is also the recipient of Nigeria's highest award for intellectual achievement, the Nigerian National Merit Award.
	At present, Mr. Achebe lives with his wife in Annandale, New York, where they both teach at Bard College. They have four children.


ALSO BY CHINUA ACHEBE 
Anthills of the Savannah 
Arrow of God 
Girls at War and Other Stories 
A Man of the People 
No Longer at Ease 

Nonfiction: 
Hopes and Impediments: Selected Essays 
The Trouble With Nigeria 

Poetry: 
Beware Soul Brother



The above code shows the last 1000 characters of the combined dataset. When we check this, they belong to Things Fall Apart. This shows that Things Fall Apart was the last one to be concatenated.

In [ ]:
# Split the combined text into sentences
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', combined_text)

# Print the last 10 sentences
for sentence in sentences[-10:]:
    print(sentence)

He was appointed Senior Research Fellow at the University of Nigeria, Nsukka, and began lecturing widely abroad.
	From 1972 to 1976, and again in 1987 to 1988, Mr. Achebe was Professor of English at the University of Massachusetts, Amherst, and also for one year at the University of Connecticut, Storrs.
	Cited in the London Sunday Times as one of the "1,000 Makers of the Twentieth Century" for defining "a modern African literature that was truly African" and thereby making "a major contribution to world literature," has published novels, short stories, essays, and children's books.
His volume of poetry, Christmas in Biafra, written during the Biafran War, was the joint winner of the first Commonwealth Poetry Prize.
Of his novels, Arrow of God is winner of the New Statesman-Jock Campbell Award, and Anthills of the Savannah was a finalist for the 1987 Booker Prize in England.
	Mr. Achebe has received numerous honours from around the world, including the Honorary Fellowship of the America

The last ten sentences.

In [ ]:
# Number of characters
num_characters = len(combined_text)

# Number of words
words = combined_text.split()
num_words = len(words)
num_words

275041

This is the number of words in the combined text.

In [ ]:
# Number of characters
num_characters

1493656

# Preprocessing

We clean and tokenize the text. This involves removing any special characters that are not useful for training a language model, such as punctuation and numbers, and then splitting the text into tokens (words) that the model can learn from.

In [ ]:
import nltk
nltk.download('punkt')
import nltk
nltk.download('stopwords')

import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Example preprocessing function
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Removing special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Stemming
    #stemmer = PorterStemmer()
    #tokens = [stemmer.stem(word) for word in tokens]

    # Join tokens back into text
    processed_text = ' '.join(tokens)

    return processed_text

# Preprocess the concatenated text data
preprocessed_data = preprocess_text(combined_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk import word_tokenize, sent_tokenize
import re
from nltk.tokenize import ToktokTokenizer
import os
import requests
import io

In [ ]:
# Tokenize the text.
tokenized_text = [list(map(str.lower, word_tokenize(sent)))
                  for sent in sent_tokenize(preprocessed_data)]

In [ ]:
#tokenized_text[0]

In [ ]:
# Preprocess the tokenized text for 3-grams language modelling
n = 3 # n gram
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

# Training an tri-gram Model

In [ ]:
# initialize MLE model - MLE model, creates an empty vocabulary
model = MLE(n)


In [ ]:
# train model
model.fit(train_data, padded_sents)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 15206 items>


In [ ]:
len(model.vocab)

15206

In [ ]:
#print(model.vocab.lookup(tokenized_text[0]))

In [ ]:
print(model.vocab.lookup('antihills savannah chinua achebe published.'.split()))

('<UNK>', 'savannah', 'chinua', 'achebe', '<UNK>')


# Exploring the model

In [ ]:
print(model.counts)

<NgramCounter with 3 ngram orders and 398778 ngrams>


In [ ]:
phrase = 'possible'
print("number of bigrams for the phrase", phrase, model.counts[phrase])

number of bigrams for the phrase possible 29


In [ ]:
phrase = 'ask'
print("number of bigrams for the phrase", phrase, model.counts[phrase])

number of bigrams for the phrase ask 171


In [ ]:
# number of bigrames of information followed by question
print("number of bigrams for the phrase", model.counts[['ask']]['question'])

number of bigrams for the phrase 3


In [ ]:
# number of bigrames of information followed by ask
print("number of bigrams for the phrase", model.counts[['question']]['ask'])

number of bigrams for the phrase 1


In [ ]:
print("number of bigrams for the phrase", model.counts[['cooking', 'food']]['kitchen'])

number of bigrams for the phrase 0


In [ ]:
# what is the probability score for the word information i.e how likely is the word in our vocanulary
# model.score('information') # P('information')
model.score("information")

0.0003009170446937041

In [ ]:
# probability of 'ask' given 'question'
model.score('ask', 'question'.split())  # P('ask'|'question') [question ask]

0.01282051282051282

In [ ]:
# probability of 'is' given 'language'
model.score('question', 'ask'.split())  # P('is'|'language') [language is]

0.017543859649122806

# Generating text

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [ ]:
detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=42):
    """Generate text method  """
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

In [ ]:
generate_sent(model, 50, random_seed=52)

'worked arranged meeting nothing came first obierikas relatives friends began arrive senior people took interest poor clarke two wives ran great alarm pleading sacred week okonkwo man thought things like colours food behaviour liked didnt like would live see desecrated okeke interpreted wisely spirits leaders umuofia asking meet headquarters also'

We can generate the above text. It is in form of tokens so it does not make much sense.

# Evaluating the model

In [ ]:
perplexity=(1/model.score("man"))**(1/3)
perplexity

4.8405554251185805

The perplexity is high at 4.84 which indicates low performance and lower confidence of th model.

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

# Reference text from the book
reference = ["school teach young christians"]

# lets see how well the model can predict the word University
candidate = "school teach young "

# Tokenize the reference and candidate sentences
reference_tokens = [sentence.split() for sentence in reference]
candidate_tokens = candidate.split()

# Calculate BLEU score
bleu_score = sentence_bleu(reference_tokens, candidate_tokens)
print(f'BLEU Score: {bleu_score}')

BLEU Score: 8.751273976943354e-78


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


The low BLEU score indicates low performance of the model.

# Training LSTM model

We split this dataset into training, validation, and test sets. This is important for training and evaluating our language model effectively.This split ensures that we have separate datasets for training the model, tuning its parameters, and finally evaluating its performance.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the concatenated text into train and test sets
train_text, test_text = train_test_split(preprocessed_data, test_size=0.2, random_state=42)

# Split the train text into train and validation sets
train_text, val_text = train_test_split(train_text, test_size=0.1, random_state=42)

# Print the sizes of the train, validation, and test sets
print("Train set size:", len(train_text))
print("Validation set size:", len(val_text))
print("Test set size:", len(test_text))

Train set size: 643877
Validation set size: 71542
Test set size: 178855


In [ ]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from nltk.translate.bleu_score import corpus_bleu
from nltk.tokenize import word_tokenize

In [ ]:
m# Define parameters
vocab_size = 10000  # Adjust according to your vocabulary size
max_sequence_length = 50  # Adjust according to your desired sequence length
embedding_dim = 100  # Adjust according to your embedding dimension
batch_size = 64
epochs = 10

In [ ]:
# Tokenize the training data
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_text)

In [ ]:
# Convert text sequences to integer sequences
train_sequences = tokenizer.texts_to_sequences(train_text)
val_sequences = tokenizer.texts_to_sequences(train_text)

In [ ]:
# Pad sequences to a fixed length
train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post')
val_padded = pad_sequences(val_sequences, maxlen=max_sequence_length, padding='post')

In [ ]:
# Prepare data for the LSTM model
x_train = train_padded[:, :-1]
y_train = train_padded[:, -1]
x_val = val_padded[:, :-1]
y_val = val_padded[:, -1]

In [ ]:
# Define the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length-1))
model.add(LSTM(256))
model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

Epoch 1/10
10061/10061 [==============================] - 134s 13ms/step - loss: 0.0106 - accuracy: 0.9999 - val_loss: 5.9860e-07 - val_accuracy: 1.0000
Epoch 2/10
10061/10061 [==============================] - 167s 17ms/step - loss: 1.1479e-07 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
10061/10061 [==============================] - 172s 17ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
10061/10061 [==============================] - 175s 17ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
10061/10061 [==============================] - 168s 17ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
10061/10061 [==============================] - 168s 17ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
10061/10061 [==============================] - 168s

The model achieves perfect accuracy (100%) and a very low loss on both the training and validation datasets, indicating that it can perfectly predict the outcomes for the given data, this might indicate suggest overfitting.

In [ ]:
# Evaluate the model using perplexity
val_loss = model.evaluate(x_val, y_val)[0]
perplexity = np.exp(val_loss)
print("Perplexity:", perplexity)

20122/20122 [==============================] - 93s 5ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Perplexity: 1.0


The perplexity of the model is 1, indicating good performance.

In [ ]:
def generate_text(seed_text, max_sequence_length, model, tokenizer):
    generated_text = seed_text
    for _ in range(50):  # Adjust the number of words to generate
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen=max_sequence_length-1, padding='pre')
        y_pred = np.argmax(model.predict(encoded), axis=-1)
        if y_pred[0] == 0:  # Check if predicted index is 0
            break  # Exit the loop if predicted index is 0
        predicted_word = tokenizer.index_word.get(y_pred[0], '')  # Use get method to handle KeyError
        if not predicted_word:
            break  # Exit the loop if predicted_word is empty
        generated_text += ' ' + predicted_word
        seed_text += ' ' + predicted_word
    return generated_text

In [ ]:
# Evaluate the model using BLEU metric
def evaluate_bleu(reference_text, generated_text):
    reference_corpus = [word_tokenize(sentence.lower()) for sentence in reference_text]
    generated_corpus = [word_tokenize(sentence.lower()) for sentence in generated_text]
    return corpus_bleu(reference_corpus, generated_corpus)


In [ ]:
# Example evaluation using test set
test_sequences = tokenizer.texts_to_sequences(test_text)
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')
x_test = test_padded[:, :-1]
y_test = test_padded[:, -1]


The perplexity and the accuracy indicate that the model is doing well. The low loss also shows that the model is doing well.

# Recurrent Neural Network



In [ ]:
from nltk.tokenize import RegexpTokenizer
import numpy as np

In [ ]:
#We select the first 100,000 elements of the preprocessed data.
partial_text = preprocessed_data[:100000]

In [ ]:
# Tokenize the partial_text into words
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [ ]:
# Find the unique tokens and create a dictionary mapping tokens to their index
unique_tokens = np.unique(tokens)
unique_token_index = {token: index for index, token in enumerate(unique_tokens)}

In [ ]:
# Prepare input sequences of n_words tokens and their corresponding next word
n_words = 10
input_words = []
next_word = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_word.append(tokens[i + n_words])

In [ ]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)  # for each sample, n input words and then a boolean for each possible next word
y = np.zeros((len(next_word), len(unique_tokens)), dtype=bool)  # for each sample a boolean for each possible next word

In [ ]:
# Convert input sequences and next words into one-hot encoding
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_word[i]]] = 1

In [ ]:
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import RMSprop


In [ ]:
# Define the LSTM model architecture
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [ ]:
# Define the optimizer and compile the model
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
# Train the model and store the training history
history = model.fit(X, y, batch_size=128, epochs=10, shuffle=True).history

Epoch 1/10
112/112 [==============================] - 60s 460ms/step - loss: 8.0795 - accuracy: 0.0069
Epoch 2/10
112/112 [==============================] - 50s 444ms/step - loss: 7.9176 - accuracy: 0.0081
Epoch 3/10
112/112 [==============================] - 47s 422ms/step - loss: 7.8891 - accuracy: 0.0079
Epoch 4/10
112/112 [==============================] - 49s 438ms/step - loss: 7.8656 - accuracy: 0.0084
Epoch 5/10
112/112 [==============================] - 48s 429ms/step - loss: 7.8213 - accuracy: 0.0075
Epoch 6/10
112/112 [==============================] - 47s 418ms/step - loss: 7.7424 - accuracy: 0.0081
Epoch 7/10
112/112 [==============================] - 45s 404ms/step - loss: 7.6327 - accuracy: 0.0081
Epoch 8/10
112/112 [==============================] - 50s 451ms/step - loss: 7.5110 - accuracy: 0.0098
Epoch 9/10
112/112 [==============================] - 47s 421ms/step - loss: 7.3475 - accuracy: 0.0114
Epoch 10/10
112/112 [==============================] - 47s 416ms/step - l

   The loss is high but it is slightly decreasing with each epoch suggesting that the model's predictions are far from the actual values but it is adjusting its weights and improving its predictions over time.
   The accuracy is very low but it increases slightly with each iteration meaning that the model's predictions are incorrect for a significant portion of the data but the model is getting better at correctly classifying the data as training progresses.

In [ ]:
# Train the model on the input data X and target labels y with specified batch size, epochs, and shuffling the data
# The verbose=0 argument suppresses the output during training, and the .history attribute stores the training history
history = model.fit(X, y, batch_size=128, epochs=5, shuffle=True, verbose=0).history


In [ ]:
import pickle

In [ ]:
# Save the trained model to a file named "text_gen_model2.h5"
model.save("text_gen_model2.h5")

# Save the training history to a file named "history2.p" using pickle
with open("history2.p", "wb") as f:
    pickle.dump(history, f)



/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model


In [ ]:
model = load_model("text_gen_model2.h5")
history = pickle.load(open("history2.p", "rb"))

In [ ]:
# Define a function to predict the next word(s) given an input text and the number of best predictions to return
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1

    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [ ]:
possible = predict_next_word(" serious situation ", 5)

1/1 [==============================] - 2s 2s/step


In [ ]:
for idx in possible:
    print(unique_tokens[idx])

coming
mouth
bad
reject
found


The above are tokens corresponding to the indices.

The following function generates words after a certain phrase.

In [ ]:
# Define a function to generate text given an input text, number of words to generate, and creativity level
def generate_text(input_text, n_words, creativity=3):
    word_sequence = input_text.split()  # Split the input text into a sequence of words
    current = 0  # Initialize a variable to keep track of the current position in the word sequence
    for _ in range(n_words):
        # Create a sub-sequence of n_words from the current position in the word sequence
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            # Predict the next word(s) based on the sub-sequence and choose one of the predictions
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            # If an exception occurs (e.g., no predictions), choose a random word from the vocabulary
            choice = random.choice(unique_tokens)
        # Append the chosen word to the word sequence and move the current position by 1
        word_sequence.append(choice)
        current += 1
    # Return the generated text as a single string
    return " ".join(word_sequence)


In [ ]:
#Generate some text after a phrase
generate_text("market place", 10, 10)

1/1 [==============================] - 0s 32ms/step


'market place round young turned might things colleagues close sam called girl'

The above generated phrase has no meaning, this might be because the text used is preprocessed so it has no stopwords or punctuation or because the model is not doing well.

In [ ]:
generate_text("white man", 100, 10)

1/1 [==============================] - 0s 36ms/step


'white man taxi man school completely bad never see sir said sit years days girl girl mm god still second chair college chief look john miss story may right next dinner wait also things thought called first first behind days time quite years english reason called even today light english change clearly bad years quite two car seat lets could perhaps hands alone left back perhaps called found perhaps world general sam girl could left feet four left wonder still said found behind could called mouth see thats might wait car couldnt mouth come said two found humour times last sure give'

Just like the previous text, this also does not seem to make sense, it is just a list of tokens.